# Monte-Carlo Simulation

In [1]:
import matplotlib.pyplot as plt
import numpy as np

## Problem Desciption
The prior probalibity and likelihoods are given by 
$$p(Y=0|Y=1)=\frac{1}{2}$$
$$p(X=x|Y=0)=2-2x$$
$$p(X=x|Y=1)=2x$$

To use inverse transform sampling, we need to get the CDF(Cumulative Distribution Function) under the condition of $Y=0$ and $Y=1$ respectively.
$$\int_0^1{p(X=x|Y=0)}$$

In [10]:
Ys = np.random.randint(low=1, high=3, size=10)
print(Ys)

[2 1 2 2 1 2 1 1 1 2]


## This is LaTeX math formula test
$$a + b = c$$
$a+b=c$